## Setup Environment

In [1]:
%%capture
%pip install langchain
%pip install langchain-hub
%pip install langchain-community langchain-huggingface
%pip install huggingface_hub transformers
%pip install sentence_transformers==2.2.2
%pip install chromadb faiss accelerate
%pip install -U bitsandbytes
%pip install tiktoken python-dotenv
%pip install faiss-gpu
%pip install InstructorEmbedding docarray
%pip install langchain_experimental
%pip install wikipedia
%pip install numexpr

## Modules

In [2]:
# llm modules
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from transformers import GenerationConfig
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

# prompt modules
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import SystemMessage

import warnings

warnings.filterwarnings("ignore")

## Prompt Template

In [12]:
prompt = ChatPromptTemplate(
    input_variables=['agent_scratchpad', 'input'],
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                partial_variables={
                    'tools':(
                        'wikipedia - A wrapper around Wikipedia. Useful for general questions about people, places, companies, facts, historical events, or other subjects. '
                        'Input should be a search query.\n'
                        'Calculator - Useful for math-related questions.'
                    ),
                    'tool_names': 'wikipedia, Calculator'},
                input_variables=[],
                template=(
                    '<s>[INST]<<SYS>>Answer the following questions as best you can. You have access to the following tools:\n\n'
                    '{tools}\n\n'
                    'The way you use the tools is by specifying a json blob.\n'
                    'Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).\n\n'
                    'The only values that should be in the "action" field are: {tool_names}\n\n'
                    'The $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions.'
                    'Here is an example of a valid $JSON_BLOB:\n\n'
                    '```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\n'
                    'ALWAYS use the following format:\n\n'
                    'Question: the input question you must answer\n'
                    'Thought: you should always think about what to do\n'
                    'Action:\n```\n$JSON_BLOB\n```\n'
                    'Observation: the result of the action\n... (this Thought/Action/Observation can repeat N times)\n'
                    'Thought: I now know the final answer\n'
                    'Final Answer: the final answer to the original input question\n\n'
                    'Begin! Reminder to always use the exact characters `Final Answer` when responding.<</SYS>>'
                )
            )
        ),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['agent_scratchpad', 'input'],
                template='{input}\n\n{agent_scratchpad}[/INST]'
                )
        )
    ]
)

In [13]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], partial_variables={'tools': 'wikipedia - A wrapper around Wikipedia. Useful for general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\nCalculator - Useful for math-related questions.', 'tool_names': 'wikipedia, Calculator'}, template='<s>[INST]<<SYS>>Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).\n\nThe only values that should be in the "action" field are: {tool_names}\n\nThe $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions.Here is an example of a valid $JSON_BLOB:\n\n```

## LLM

In [5]:
model_name = "minkhantycc/Llama-2-7b-chat-finetune-quantized"
# model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
device_map = {"": 0}

# bnb config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=False,
)

# base model
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
base_model.config.use_cache = False

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model.generation_config = GenerationConfig(
    max_new_tokens = 256,
    temperature = 0.00000001,
    repetition_penalty = 1.15,
    do_sample = False,
    eos_token_id = tokenizer.eos_token_id,
    pad_token_id = tokenizer.eos_token_id,
)

# pipeline
pipe = pipeline(
    task="text-generation",
    model=base_model,
    tokenizer=tokenizer,
    device_map=device_map,
    return_full_text=False
)

config.json:   0%|          | 0.00/630 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [6]:
# llm
hf_pipe = HuggingFacePipeline(pipeline=pipe).bind(stop=["\nObservation:", "\n\tObservation:", 'Question:', '</s>', '\n\n'])

## Built-in Tools

In [7]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_experimental.utilities.python import PythonREPL

In [14]:
## math and wikipedia tools
tools = load_tools(["llm-math", "wikipedia"], llm=hf_pipe)

# initialize agent
agent = initialize_agent(
    tools,
    hf_pipe,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)
agent.agent.llm_chain.prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Answer the following questions as best you can. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math.\nwikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).\n\nThe only values that should be in the "action" field are: Calculator, wikipedia\n\nThe $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_

In [15]:
# reformat the prompt
agent.agent.llm_chain.prompt = prompt
agent.agent.llm_chain.prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], partial_variables={'tools': 'wikipedia - A wrapper around Wikipedia. Useful for general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\nCalculator - Useful for math-related questions.', 'tool_names': 'wikipedia, Calculator'}, template='<s>[INST]<<SYS>>Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).\n\nThe only values that should be in the "action" field are: {tool_names}\n\nThe $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions.Here is an example of a valid $JSON_BLOB:\n\n```

In [16]:
import langchain
langchain.debug=True

# math agent test
result = agent("What is the 25% of 300?")
langchain.debug=False
result

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What is the 25% of 300?"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "What is the 25% of 300?",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:HuggingFacePipeline] Entering LLM run with input:
{
  "prompts": [
    "System: <s>[INST]<<SYS>>Answer the following questions as best you can. You have access to the following tools:\n\nwikipedia - A wrapper around Wikipedia. Useful for general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\nCalculator - Useful for math-related questions.\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).\n\nThe only values tha

{'input': 'What is the 25% of 300?',
 'output': '75\n\nYou may want to consider using a calculator to perform calculations involving percentages.'}

In [17]:
# wikipedia agent test
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question)
result



> Entering new AgentExecutor chain...
Could not parse LLM output:  The book written by Tom M. Mitchell is "Artificial Intelligence: A Modern Approach". 
Observation: Invalid or incomplete response
Thought:Tom M. Mitchell wrote the book "Artificial Intelligence: A Modern Approach".

Final Answer: Artificial Intelligence: A Modern Approach

Please provide more information on how to proceed with your request. 

> Finished chain.


{'input': 'Tom M. Mitchell is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU)what book did he write?',
 'output': 'Artificial Intelligence: A Modern Approach\n\nPlease provide more information on how to proceed with your request.'}

## Python Agent

In [51]:
# python agent
agent = create_python_agent(
    hf_pipe,
    tool=PythonREPLTool(),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_executor_kwargs = {"handle_parsing_errors":True}
)

agent.agent.llm_chain.prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input'], template='You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an error, debug your code and try again.\nOnly use the output of your code to answer the question. \nYou might know the answer without running any code, but you should still run the code to get the answer.\nIf it does not seem like you can write code to answer the question, just return "I don\'t know" as the answer.\n\n\nPython_REPL - A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Python_REPL]\nAction Input: the input to the action\nObservation: the

In [42]:
python_prompt = PromptTemplate(
    input_variables=['agent_scratchpad', 'input'],
    partial_variables={
        'tool': "Python REPL - A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
        'tool_name': 'Python REPL'
        },
    template=(
        '<s>[INST]<<SYS>>You are an agent designed to write and execute python code to answer questions.\n'
        'You have access to {tool_name}, which you can use to execute python code.\n'
        'If you get an error, debug your code and try again.\n'
        'Only use the output of your code to answer the question. \n'
        'You might know the answer without running any code, but you should still run the code to get the answer.\n'
        'If it does not seem like you can write code to answer the question, just return "I don\'t know" as the answer.\n\n\n'
        '{tool_name} - {tool}\n\n'
        'Use the following format:\n\n'
        'Question: the input question you must answer\n'
        'Thought: you should always think about what to do\n'
        'Action: the action to take, should be one of [tool_name]\n'
        'Action Input: the input to the action\n'
        'Observation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\n'
        'Thought: I now know the final answer\n'
        'Final Answer: the final answer to the original input question\n\n'
        'Begin!\n\n<</SYS>>'
        'Question: {input}\nThought:{agent_scratchpad}[/INST]'
    )
)

In [48]:
# agent.agent.llm_chain.prompt = python_prompt
agent.agent.llm_chain.prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tool': 'Python REPL - A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.', 'tool_name': 'Python REPL'}, template='<s>[INST]<<SYS>>You are an agent designed to write and execute python code to answer questions.\nYou have access to {tool_name}, which you can use to execute python code.\nIf you get an error, debug your code and try again.\nOnly use the output of your code to answer the question. \nYou might know the answer without running any code, but you should still run the code to get the answer.\nIf it does not seem like you can write code to answer the question, just return "I don\'t know" as the answer.\n\n\n{tool_name} - {tool}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to

In [53]:
# input to python
customer_list = [
    ["Harrison", "Chase"],
    ["Lang", "Chain"],
    ["Dolly", "Too"],
    ["Elle", "Elem"],
    ["Geoff","Fusion"],
    ["Trance","Former"],
    ["Jen","Ayai"]
]

langchain.debug=True
result = agent.run(
    f"""Sort these custormers by last name and then first name \
    and print the output: {customer_list}"""
)
langchain.debug=False
result

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these custormers by last name and then first name     and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these custormers by last name and then first name     and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:HuggingFacePipeline] Entering LLM run with input:
{
  "prompts": [
    "You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nI

'The list of customers sorted alphabetically by last name and then first name.'